In [ ]:
import sys
import inspect
from datetime import datetime, timedelta

# Add pub_worm directory to the Python path
sys.path.insert(0, "/Users/dan/Code/Python/pub_worm")

from pub_worm.ncbi.entreze_api import EntrezAPI
from pub_worm.biorxiv.biorxiv_api import biorxiv_search

# Find where EntrezAPI is being load from
module = inspect.getmodule(EntrezAPI)
if hasattr(module, "__file__"):
    file_path = module.__file__
    print("EntrezAPI imported from:", file_path)
else:
    print("Could not determine the file path.")

In [ ]:
!pwd


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the slack_hook variable
slack_webhook = os.getenv('slack_webhook')


In [ ]:
term = "C. elegans"
days=0
# Get the date range for the search (last `days` days)
today = datetime.today()
start_date = (today - timedelta(days=days)).strftime('%Y/%m/%d')
end_date = today.strftime('%Y/%m/%d')

search_term = f"{term} AND ({start_date}[PDAT] : {end_date}[PDAT])"

ncbi_api = EntrezAPI()
esearch_params = {'term': search_term }
esearch_result = ncbi_api.entreze_esearch(esearch_params)
#epost_result = ncbi_api.entreze_epost(esearch_result)
efetch_result = ncbi_api.entreze_efetch(esearch_result)
for entry in efetch_result['articles']:
    print(f"Title: {entry['title']}")
    print(f"doi: https://doi.org/{entry['doi']}")
    print()
    post_to_slack(entry['title'], f"https://doi.org/{entry['doi']}", slack_webhook)

In [ ]:

result_dict = biorxiv_search(from_date="2024-10-23", to_date="2024-10-24")
for entry in result_dict:
    print(entry)

In [ ]:
import requests
import json

def post_to_slack(new_title, new_url, slack_webhook):
    slack_webhook_url=f"https://hooks.slack.com/services/{slack_webhook}"
    message = f"*{new_title}*\n{new_url}"
    slack_data = {
        "blocks": [
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": message
                }
            }
        ]
    }

    response = requests.post(slack_webhook_url, data=json.dumps(slack_data), headers={'Content-Type': 'application/json'})
    
    if response.status_code != 200:
        raise Exception(f"Slack API request failed with status code {response.status_code}")



In [ ]:
result_dict = biorxiv_search()
for entry in result_dict:
    print(entry)
    

In [ ]:
post_to_slack(result_dict[0], slack_webhook)